# 🎰 HubSpot Dummy Data Generator

After registering a new account in the HubSpot, it contains almost no data, so the reports eventually have no figures.

This notebook generates dummy data for companies, contacts, deals, tickets, and line items as a downloadable Excel file.

Once the data has been imported to the HubSpot, the following reports can be generated:
* Closed revenue by month with deal total and closed revenue breakdown
* Closed deal revenue amount by source over time
* Deal stage weighted revenue per day vs. goal
* Deal created totals vs. goal
* Company revenue by close date with source breakdown
* Ticket totals by status

Run all the cells in this notebook to create a downloadable Excel file for import into the HubSpot.

## 1. Install Required Packages

Install all necessary packages for data generation.

In [ ]:
%pip install faker pandas openpyxl

import base64
from datetime import datetime, timedelta
from faker import Faker
import io
from IPython.display import HTML, display
import pandas as pd

## 2. Configuration - Edit These Values

Modify the values below to customize your data generation.

### 2.1 Countries Selection

Select which countries you want to generate data for. Available options:
- Canada
- France
- Germany
- Italy
- Japan
- United Kingdom
- United States
- Austria
- Switzerland

In [ ]:
# Define the list of countries for which we want to generate data
countries = ["Germany"]

### 2.2 Data Volume Settings

Define how many records you want to generate.
- **Total records** = count(countries) × company_rows × contacts_deals_tickets_per_company

In [ ]:
# Define how many records you want in the file
company_rows = 6
contacts_deals_tickets_per_company = 10

### 2.3 Deal Settings

Define the deal's _close date_ range starting from today and back to `oldest_deal_days_ago`, this will affect the X axis in the reports.

The deal's create date will be in the range of `deal_min_length_days` to `deal_max_length_days` days before the _close date_.

Specify the number of deals you want closed and of which the number of deals won with the appropriate parameters below.

In [ ]:
# Define deal dates parameters
oldest_deal_days_ago = 90
deal_min_length_days = 7
deal_max_length_days = 21

# Define deal closure parameters
closed_deals_percentage = 70
won_deals_percentage = 32

### 2.4 HubSpot Constants

The following values should match the ones defined in your HubSpot's [data model](https://knowledge.hubspot.com/data-management/view-a-model-of-your-crm-object-and-activity-relationships#use-the-data-model-overview).

In [ ]:
# Define properties possible values
company_industries = ["INFORMATION_TECHNOLOGY_AND_SERVICES", "HOSPITAL_HEALTH_CARE", "FINANCIAL_SERVICES", "COMMERCIAL_REAL_ESTATE"]

deal_stages = ["appointmentscheduled", "qualifiedtobuy", "presentationscheduled", "decisionmakerboughtin"]
deal_stage_closed_won = "closedwon"
deal_stage_closed_lost = "closedlost"

deal_types = ["newbusiness", "existingbusiness"]
traffic_sources = ["DIRECT_TRAFFIC", "ORGANIC_SEARCH", "PAID_SEARCH", "SOCIAL_MEDIA", "PAID_SOCIAL"]

ticket_statuses = ["1", "2", "3", "4"]
ticket_priorities = ["LOW", "MEDIUM", "HIGH", "URGENT"]

# Define Excel columns as HubSpot objects properties to import to
column_names = {
        "company_name": "<COMPANY name>",
        "company_domain": "<COMPANY domain>",
        "company_industry": "<COMPANY industry>",
        "company_address": "<COMPANY address>",
        "company_country": "<COMPANY country>",
        "contact_first_name": "<CONTACT firstname>",
        "contact_last_name": "<CONTACT lastname>",
        "contact_email": "<CONTACT email>",
        "contact_analytics_source": "<CONTACT hs_analytics_source>",
        "contact_phone": "<CONTACT phone>",
        "contact_address": "<CONTACT address>",
        "contact_country": "<CONTACT country>",
        "contact_jobtitle": "<CONTACT jobtitle>",
        "contact_job_function": "<CONTACT job_function>",
        "deal_name": "<DEAL dealname>",
        "deal_stage": "<DEAL dealstage>",
        "deal_pipeline": "<DEAL pipeline>",
        "deal_amount": "<DEAL amount>",
        "deal_type": "<DEAL dealtype>",
        "deal_analytics_source": "<DEAL hs_analytics_source>",
        "deal_create_date": "<DEAL createdate>",
        "deal_close_date": "<DEAL closedate>",
        "ticket_subject": "<TICKET subject>",
        "ticket_pipeline": "<TICKET hs_pipeline>",
        "ticket_pipeline_stage": "<TICKET hs_pipeline_stage>",
        "ticket_priority": "<TICKET hs_ticket_priority>",
        "line_item_name": "<LINE_ITEM name>",
        "line_item_price": "<LINE_ITEM price>",
        "line_item_description": "<LINE_ITEM description>",
        "line_item_quantity": "<LINE_ITEM quantity>"
    }

You can set the values below as strings of your choice.

In [ ]:
# Define possible values
contacts_job_functions = ["Sales", "Marketing", "Human Resources", "Engineering"]

### 2.5 Output Filename Settings

Configure the output Excel filename.

In [ ]:
# Generate filename with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_filename = f"hubspot_dummy_data_{timestamp}.xlsx"

print(f"Output file will be: {output_filename}")

## 3. Data Generation Function

Function to generate dummy data for a specific country.

In [ ]:
def generate_records(country: str) -> pd.DataFrame:
    """
    Generate dummy records for a given country.
    
    Args:
        country (str): The country name to generate data for
    
    Returns:
        pd.DataFrame: DataFrame containing generated data
    """
    # Set the locale for Faker based on the country
    LOCALE_MAP = {
        "Canada": "en_CA",
        "France": "fr_FR",
        "Germany": "de_DE",
        "Italy": "it_IT",
        "Japan": "ja_JP",
        "United Kingdom": "en_GB",
        "United States": "en_US",
        "Austria": "de_AT",
        "Switzerland": "de_CH"
    }

    assert 0 <= closed_deals_percentage <= 100, "closed_deals_percentage must be between 0 and 100"
    assert 0 <= won_deals_percentage <= 100, "won_deals_percentage must be between 0 and 100"
    assert deal_min_length_days < deal_max_length_days, "deal_min_length_days must be less than deal_max_length_days"
    assert all(c in LOCALE_MAP for c in countries), f"Invalid country. Must be one of: {list(LOCALE_MAP.keys())}"

    locale = LOCALE_MAP.get(country, "en_US")
    faker = Faker(locale)

    def short_uuid() -> str:
        uuid = str(faker.uuid4())
        short = uuid[-7:]
        return short

    DATE_FORMAT = "%Y.%m.%d"

    def generate_single_record():
        """Generate a single company record with associated contact, deal, ticket, and line item."""
        # Company
        company_name = faker.company()
        company_domain = faker.domain_name()
        company_industry = faker.random_element(company_industries)
        company_street_address = faker.address()
        
        # Contact
        contact_first_name = faker.first_name()
        contact_last_name = faker.last_name()
        contact_email = faker.email()
        contact_analytics_source = faker.random_element(traffic_sources)
        contact_phone = faker.phone_number()
        contact_street_address = faker.address()
        contact_jobtitle = faker.job()
        contact_job_function = faker.random_element(contacts_job_functions)
        
        # Deal
        deal_name = f'Deal-{short_uuid()}'
        deal_close_date = datetime.today() - timedelta(days=faker.random_int(min=1, max=oldest_deal_days_ago))
        deal_create_date = deal_close_date - timedelta(days=faker.random_int(min=deal_min_length_days, max=deal_max_length_days))
        line_item_price = faker.random_int(min=10, max=1000)
        line_item_quantity = faker.random_int(min=1, max=100)
        deal_amount = line_item_price * line_item_quantity
        deal_pipeline = "default"
        deal_type = faker.random_element(deal_types)
        deal_analytics_source = faker.random_element(traffic_sources)
        
        is_deal_closed = faker.boolean(chance_of_getting_true=closed_deals_percentage)
        if is_deal_closed:
            is_deal_won = faker.boolean(chance_of_getting_true=won_deals_percentage)
            deal_stage = deal_stage_closed_won if is_deal_won else deal_stage_closed_lost
            deal_close_date = deal_close_date
        else:
            deal_stage = faker.random_element(deal_stages)
            deal_close_date = None
        
        # Ticket
        ticket_subject = f'Ticket-{short_uuid()}'
        ticket_pipeline = "0"
        ticket_pipeline_stage = faker.random_element(ticket_statuses)
        ticket_priority = faker.random_element(ticket_priorities)
        
        # Line item
        line_item_name = f'Lineitem-{short_uuid()}'
        line_item_description = faker.catch_phrase()
        
        # Format values
        company_address_str = company_street_address.replace('\n', ', ')
        contact_address_str = contact_street_address.replace('\n', ', ')
        deal_create_date_str = deal_create_date.strftime(DATE_FORMAT)
        deal_close_date_str = deal_close_date.strftime(DATE_FORMAT) if deal_close_date else None
        
        return {
            column_names["company_name"]: company_name,
            column_names["company_domain"]: company_domain,
            column_names["company_industry"]: company_industry,
            column_names["company_address"]: company_address_str,
            column_names["company_country"]: country,
            column_names["contact_first_name"]: contact_first_name,
            column_names["contact_last_name"]: contact_last_name,
            column_names["contact_email"]: contact_email,
            column_names["contact_analytics_source"]: contact_analytics_source,
            column_names["contact_phone"]: contact_phone,
            column_names["contact_address"]: contact_address_str,
            column_names["contact_country"]: country,
            column_names["contact_jobtitle"]: contact_jobtitle,
            column_names["contact_job_function"]: contact_job_function,
            column_names["deal_name"]: deal_name,
            column_names["deal_stage"]: deal_stage,
            column_names["deal_pipeline"]: deal_pipeline,
            column_names["deal_amount"]: deal_amount,
            column_names["deal_type"]: deal_type,
            column_names["deal_analytics_source"]: deal_analytics_source,
            column_names["deal_create_date"]: deal_create_date_str,
            column_names["deal_close_date"]: deal_close_date_str,
            column_names["ticket_subject"]: ticket_subject,
            column_names["ticket_pipeline"]: ticket_pipeline,
            column_names["ticket_pipeline_stage"]: ticket_pipeline_stage,
            column_names["ticket_priority"]: ticket_priority,
            column_names["line_item_name"]: line_item_name,
            column_names["line_item_price"]: line_item_price,
            column_names["line_item_description"]: line_item_description,
            column_names["line_item_quantity"]: line_item_quantity
        }

    # Generate records
    records = [
        generate_single_record()
        for _ in range(company_rows)
        for _ in range(contacts_deals_tickets_per_company)
    ]

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(records)
    return df

## 4. Generate Data for All Countries

Run the data generation process for all selected countries.

In [ ]:
dataset = pd.DataFrame()

print("Generating records for countries:", countries)

for country in countries:
    df = generate_records(country)
    dataset = pd.concat([dataset, df], ignore_index=True)
    print(f"  ✓ Generated {len(df)} records for {country}")

print(f"\n✅ Total records generated in the dataset: {len(dataset)}")
print(f"   Columns: {len(dataset.columns)}")

## 5. Preview Generated Data

Display a sample of the generated data and summary statistics.

In [ ]:
# Display first few rows
print("First 5 rows of generated dataset:\n")
display(dataset.head())

## 6. Download Excel File

Export the generated data to an Excel file with timestamp.

In [ ]:
# Create an in-memory Excel file
with io.BytesIO() as buffer:
    dataset.to_excel(buffer, index=False)
    buffer.seek(0)
    content = buffer.getvalue()

# Encode as base64
b64 = base64.b64encode(content).decode()
href = f'<h4>📥 <a download="{output_filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">Download {output_filename}</a></h4>'

print(f"Dataset successfully generated: {output_filename}")
print(f"   Total records: {len(dataset)}")
print(f"   File size: {len(content) / 1024:.2f} KB")
print("\n")
display(HTML(href))
print("\n")

## 7. Import the Excel file into the HubSpot

Make sure to read about [daily limits](https://knowledge.hubspot.com/import-and-export/set-up-your-import-file) for imports at HubSpot before start.

1. [Login](https://hubspot.com) to the HubSpot and open CRM - Deals page
2. Press the **Add Deals** - **Import** button in the top right corner of the screen
3. On the import page choose Import a file and press the **Start import** button
4. On the next screen select the following objects:
    - Companies
    - Contacts
    - Deals
    - Line Items
    - Tickets
4. Press the **Next** button in the bottom right corner of the screen
5. Upload the Excel file by clicking **choose a file** link in the middle of the page
6. Press the **Next** button in the bottom right corner of the screen
7. Fix potential import errors by clicking the **yellow exclimation mark icon** in the closedate row and selecting the **year month day** date format, close the view with the **X** button
8. Continue the import by pressing the **Next** button and following further instruction on the screen

---

Copyright © 2025 Ivan Rublev

This notebook is licensed under the [MIT license](https://mit-license.org/license.txt)